In [40]:
# Ingestion

def leggi_file_txt(nome_file):

    lista_righe = []
    try:
        with open(nome_file, 'r') as file:
            for riga in file:
                lista_righe.append(riga[:-1].split(' '))


    except FileNotFoundError:
        print(f"Errore: Il file {nome_file} non è stato trovato.")
    except ValueError as e:
        print(f"Errore durante la conversione dei dati: {e}")
    out=[]
    for riga in lista_righe:
        disposizione=[]
        for carattere in riga[0]:
            if carattere=='.':
                disposizione.append(0)
            elif carattere=='#':
                disposizione.append(1)
        pulsanti=[]
        for entrata in riga[1:-1]:
            pulsante=[]
            for carattere in list(entrata):
                if carattere.isdigit():
                    pulsante.append(int(carattere))
            pulsanti.append(pulsante)
        joltage=[ int(item)  for item in  riga[-1][1:-1].split(',')]
        out.append([disposizione, pulsanti, joltage])
                   
    return  out


nome_file = 'input_day10.txt'
nome_file_test= 'input_day10_test.txt'
data= leggi_file_txt(nome_file)
data_test=leggi_file_txt(nome_file_test)

In [21]:
#Funzioni parte 1

import itertools

def combinazioni(n):
    field = [0, 1] 
    out=[]
    for item in itertools.product(field, repeat=n):
        out.append(item)
    return out

def premi_pulsanti(combinazione, pulsanti, numero_luci):
    luci=[0 for i in range(numero_luci)]
    for i in range(len(combinazione)):
        if combinazione[i]==1:
                for pulsante in pulsanti[i]:
                    luci[pulsante]=(luci[pulsante]+1)%2
    return luci

def risolvi(problema):
    soluzioni=[]
    for comb in combinazioni(len(problema[1])):
        if premi_pulsanti(comb, problema[1], len(problema[0])) == problema[0]:
            soluzioni.append(sum(comb))
    return min(soluzioni)
        
        

In [22]:
#Part 1

out_1=0

input=data
for macchina in input:
    out_1+=risolvi(macchina)
    
print(out_1)



417


In [44]:
# Funzioni parte 2
from z3 import *

def trova_matrice(lista_pulsanti, termine_noto):
    A=[]
    for pulsante in lista_pulsanti:
        app=[]
        for i in range(len(termine_noto)):
            if i in pulsante:
                app.append(1)
            else:
                app.append(0)
        A.append(app)
    return A

def premi_pulsanti_unlimited(combinazione, pulsanti, numero_luci):
    luci=[0 for i in range(numero_luci)]
    for i in range(len(combinazione)):
        if combinazione[i]!=0:
                for pulsante in pulsanti[i]:
                    luci[pulsante]=luci[pulsante]+combinazione[i]
    return luci


def risolvi_diofanteo(A,b):
    n=len(A)
    m=len(b)

    x = [Int(f"x{i}") for i in range(n)]

    opt=Optimize()
    for x_i in x:
        opt.add(x_i >= 0)


    for i in range(m):
        opt.add(Sum(A[j][i]* x[j] for j in range(n))==b[i])
    
    opt.minimize(Sum(x))

    
    if opt.check() == sat:
        model = opt.model()
        return [model[xi].as_long() for xi in x]
    else:
         return None

In [47]:
#Parte 2
out_2=0

for riga in data:
    M=trova_matrice(riga[1], riga[2])
    soluzione=risolvi_diofanteo(M, riga[2])
    out_2+=sum(soluzione)

print(out_2)

33
